In [ ]:
import cv2
import numpy as np

In [ ]:
image_1 = cv2.imread("C:\\Users\\Darshan\\Downloads\\Computer Vision\\box.png", cv2.IMREAD_GRAYSCALE)
image_2 = cv2.imread("C:\\Users\\Darshan\\Downloads\\Computer Vision\\box_in_scene.png", cv2.IMREAD_GRAYSCALE)

In [ ]:
# Resize image 2 with respect to image 1
image_2 = cv2.resize(image_2, (image_1.shape[1], image_1.shape[0]))

In [ ]:
# Use SIFT to detect features
sift = cv2.SIFT_create()
keypoints_1, descriptors_1 = sift.detectAndCompute(image_1, None)
keypoints_2, descriptors_2 = sift.detectAndCompute(image_2, None)

In [ ]:
# Draw the keypoints on the first image
image_1_keypoints = cv2.drawKeypoints(image_1, keypoints_1, None, color=(0, 255, 0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
# cv2_imshow(image_1_keypoints)

In [ ]:
# Use FLANN for feature matching
index_params = dict(algorithm=0, trees=5)
search_params = dict(checks=100)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(descriptors_1, descriptors_2, k=2)

In [ ]:
# Use Lowe's Ratio to find good_matches
good_matches = []
for best, second_best in matches:
  if(best.distance < 0.75 * second_best.distance):
    good_matches.append(best)

In [ ]:
if(len(good_matches) > 10):
  src_pts = np.float32([keypoints_1[best.queryIdx].pt for best in good_matches]).reshape(-1, 1, 2)
  dst_pts = np.float32([keypoints_2[best.trainIdx].pt for best in good_matches]).reshape(-1, 1, 2)
  M, m = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
  h, w = image_1.shape
  aligned_image = cv2.warpPerspective(image_2, M, (w, h))
  cv2.imshow("Aligned Image",aligned_image)
  cv2.waitKey(0)
  cv2.destroyAllWindows()
else:
  print("Not enough matches")